In [1]:
#Theory Behind HMM (Jurafsky Notes):
""" A Markov chain is useful when we need to compute a probability for a sequence of
events that we can observe in the world. In many cases, however, the events we are
interested in may not be directly observable in the world. For example, in part-of-
speech tagging, we don’t observe part-of-speech tags in the world; we see words and have to infer the correct 
tags from the word sequence. Hence, we call the part-of-speech tags hidden because they are not observed.
"""
"""Imagine that you are a climatologist in the year 2799 studying the history of global
warming. You cannot find any records of the weather in Baltimore, Maryland, for
the summer of 2007, but you do find Jason Eisner’s diary, which lists how many ice
creams Jason ate every day that summer. Our goal is to use these observations to
estimate the temperature every day. We’ll simplify this weather task by assuming
there are only two kinds of days: cold (C) and hot (H). So the Eisner task is as
follows:
Given a sequence of observations O, each observation an integer cor-
responding to the number of ice creams eaten on a given day, figure
out the correct ‘hidden’ sequence Q of weather states (H or C) which
caused Jason to eat the ice cream."""
""""""

''

In [4]:
#Terminology
""" 
Q = q1 q2 . . . qN             : Set of N states
A = a[i][j] s.t. 1<=i,j<=N     : Transition probability matrix A,s.t a[i][j] is prob of moving from state i to j
a[i][j]=P(qj|qi)
pi = pi[1],pi[2],...,pi[N]     : Initial prob distr over states. pi[i] is prob of Markov chain starting in state i.
O = o[1]o[2]...O[T]            : A Sequence of T Observations, drawn from a Vocab V=v[1]v[2]...,v[v]
"""
""""""

''

In [5]:
"""A first-order hidden Markov model instantiates two simplifying assumptions.
First, as with a first-order Markov chain, the probability of a particular state depends
only on the previous state:
    Markov Assumption: P(qi |q1 ...qi−1) = P(qi|qi−1)
Second, the probability of an output observation o[i] depends only on the state that
produced the observation qi and not on any other states or any other observations:
    Output Independence: P(o[i]|q1 . . qi , . . , qT , o[1] , . . , o[i] , . . , o[T])= P(o[i]|qi)"""
""""""

''

In [6]:
"""There is a (non-zero) probability of transitioning between any two states. Such an HMM is called 
a fully connected or ergodic HMM. Sometimes, however, we have HMMs in which many of the transitions 
between states have zero probability."""
""""""

''

In [7]:
"""hidden Markov models should be characterized by three fundamental problems:
    Problem 1 (Likelihood): Given an HMM λ = (A, B) and an observation sequence O, determine the 
    likelihood P(O|λ ).
    Problem 2 (Decoding): Given an observation sequence O and an HMM λ =(A, B), discover the best hidden state 
    sequence Q.
    Problem 3 (Learning): Given an observation sequence O and the set of states in the HMM, learn the HMM 
    parameters A and B."""
""""""

''

In [8]:
import numpy as np
import math
from math import log10
import sys

In [9]:
#Likelihood Computation: The Forward Algorithm  O(N*N*T)
#Our first problem is to compute the likelihood of a particular observation sequence say 3 1 3?
"""For a Markov chain, where the surface observations are the same as the hidden
events, we could compute the probability of 3 1 3 just by following the states labeled
3 1 3 and multiplying the probabilities along the arcs. For a hidden Markov model,
things are not so simple. We want to determine the probability of an ice-cream
observation sequence like 3 1 3, but we don’t know what the hidden state sequence
is!"""
#O: Observations, a:State Transition Prob, b:Emission Prob, pi: Initial Prob. 
#a[i][j]: Prob of transition from state i to state j
#b[i][j]: prob of emitting observation j at state i
#pi[i]: Initial Prob.of state i
#P(O|a,b,pi)=P(O|X=x1,a,b,pi)+P(O|X=x2,a,b,pi)+......+P(O|X=xn,a,b,pi) s.t. x1,x2,....,xn are all possible sequences
#alpha[t][i]:Partial Observation sequence upto time t is generated and we are state i
#alpha[t][i]= alpha[t-1][0]*a[0][i]*b[i][t]+
#    alpha[t-1][1]*a[1][i]*b[i][t]+....+alpha[t-1][n-1]*a[n-1][i]*b[i][t]#Assuming we have n states
def computeAlpha(observations,a,b,pi,alphaDP):
    statesC=a.shape[0]
    timePts=observations.shape[0]
    if timePts<1:
        return
    for i in np.arange(statesC):
        alphaDP[0][i]=-(log10(pi[i])+log10(b[i][observations[0]]))#pi[i]*b[i][observations[0]]
    for t in np.arange(1,timePts):
        for i in np.arange(statesC):
            for j in np.arange(statesC):
                alphaDP[t][i]=alphaDP[t][i]-(log10(alphaDP[t-1][j])+log10(a[j][i])+log10(b[i][observations[t]]))
                #alphaDP[t][i]+=alphaDP[t-1][j]*a[j][i]*b[i][observations[t]]
def observationProb(observations,a,b,pi,alphaDP):
    computeAlpha(observations,a,b,pi,alphaDP)
    print(alphaDP)
    timePts=observations.shape[0]
    stateC=a.shape[0]
    ans=0.0
    for i in np.arange(stateC):
        ans+=alphaDP[timePts-1][i]
    return ans

In [11]:
observations=np.array([0,1,0,2])
A=np.array([[0.7,0.3],[0.4,0.6]])
B=np.array([[0.1,0.4,0.5],[0.7,0.2,0.1]])
pi=np.array([0.6,0.4])
alphaDP=np.zeros(shape=(observations.shape[0],A.shape[0]))# Count_of_Observations*Count_of_Hidden_States
#Can Easily Optimize Space Requirement of above alphaDP; Only need 2*count_of_Hidden_States Space
observationProb(observations,A,B,pi,alphaDP)

[[ 1.22184875  0.55284197]
 [ 1.51910353  2.31304905]
 [ 2.00706975  0.5087592 ]
 [ 1.14582722  2.73565275]]


3.8814799734520711

In [12]:
"""Problem 2 (Decoding): The Viterbi Algorithm -
Decoding: Given as input an HMM λ = (A, B, pi) and a sequence of observations O = o1 , o2 , ..., oT
find the most probable sequence of states Q = q1 q2 q3 . . . qT 
"""
"""Note that the Viterbi algorithm is identical to the forward algorithm except that it takes the max over the
previous path probabilities whereas the forward algorithm takes the sum. Note also
that the Viterbi algorithm has one component that the forward algorithm doesn’t have: backpointers. The reason 
is that while the forward algorithm needs to produce an observation likelihood, the Viterbi algorithm must 
produce a probability and also the most likely state sequence. We can compute this best state sequence by keeping
track of the path of hidden states that led to each state, and then at the end backtracing 
the best path to the beginning (the Viterbi backtrace).
"""
def ViterbiAlgo(observations,a,b,pi,viterbiDP,viterbiBackPtr):
    statesC=a.shape[0]
    timePts=observations.shape[0]
    if statesC<1:
        return
    for i in np.arange(statesC):
        viterbiDP[0][i]=-(log10(pi[i])+log10(b[i][observations[0]]))#pi[i]*b[i][observations[0]]
    #Doesn't Handle the Case when there is single observation(1 Time Point)
    for t in np.arange(1,timePts):
        for i in np.arange(statesC):
            maxSoFar=0
            maxJ=-1
            for j in np.arange(statesC):
                tempVal=-(log10(viterbiDP[t-1][j])+log10(a[j][i]))#viterbiDP[t-1]*a[j][i]
                if tempVal>maxSoFar:
                    maxSoFar=tempVal
                    maxJ=j
            viterbiDP[t][i]=tempVal-log10(b[i][observations[t]])#At State i,Produce observation b[i][observations[t]]
            viterbiBackPtr[t][i]=maxJ
def getOptimalHiddenStates(observations,a,b,pi,viterbiDP,viterbiBackPtr):
    ViterbiAlgo(observations,a,b,pi,viterbiDP,viterbiBackPtr)
    print(viterbiDP)
    print(viterbiBackPtr)
    timePts=observations.shape[0]
    stateC=a.shape[0]
    ans=0.0
    endState=-1
    for i in np.arange(stateC):
        if ans<alphaDP[timePts-1][i]:
            ans=alphaDP[timePts-1][i]
            endState=i
    stateSeq=np.zeros(shape=(timePts))
    stateSeq[timePts-1]=endState#This is where we are adding the Final Optimal State for the Given Observation
    i=timePts-2
    prevState=endState
    while i>=0:
        prevState=viterbiBackPtr[i+1][prevState]
        stateSeq[i]=prevState
        i=i-1
    return {"Probability":ans,"Sequence":stateSeq}

In [13]:
observations=np.array([0,1,0,2])
A=np.array([[0.7,0.3],[0.4,0.6]])
B=np.array([[0.1,0.4,0.5],[0.7,0.2,0.1]])
pi=np.array([0.6,0.4])
viterbiDP=np.zeros(shape=(observations.shape[0],A.shape[0]))# Count_of_Observations*Count_of_Hidden_States
viterbiBackPtr=np.zeros(shape=(observations.shape[0],A.shape[0]))# Count_of_Observations*Count_of_Hidden_States
getOptimalHiddenStates(observations,A,B,pi,viterbiDP,viterbiBackPtr)

[[ 1.22184875  0.55284197]
 [ 1.05327901  1.17821775]
 [ 1.32671445  0.30552515]
 [ 1.21392304  1.73680179]]
[[ 0.  0.]
 [ 1.  1.]
 [ 1.  0.]
 [ 1.  1.]]


/home/alok/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:48: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


{'Probability': 2.735652754181511, 'Sequence': array([ 1.,  0.,  1.,  1.])}

In [1]:
"""Problem 3 (HMM Parameter Training): The Forward-Backward/Baum-Welch Algorithm(Special Case of EM Algo) -
Given an observation sequence O and the set of possible states in the HMM, learn the HMM 
parameters λ = (A, B) 
"""
"""The input to such a learning algorithm would be an unlabeled sequence of ob-
servations O and a vocabulary of potential hidden states Q. Thus, for the ice cream
task, we would start with a sequence of observations O = {1, 3, 2, ..., } and the set
of hidden states H and C. For the part-of-speech tagging task, we would start with
a sequence of observations O = {w1 , w2 , w3 . . .} and a set of hidden states NN, NNS,
VBD, IN,... and so on.
"""
"""
Let us begin by considering the much simpler case of training a Markov chain rather than a hidden Markov model. Since 
the states in a Markov chain are observed, we can run the model on the observation sequence and directly see which path 
we took through the model and which state generated each observation symbol. A Markov chain of course has no emission 
probabilities B (alternatively, we could view a Markov chain as a degenerate hidden Markov model where all the b proba-
bilities are 1.0 for the observed symbol and 0 for all other symbols). Thus, the only probabilities we need to train are 
the transition probability matrix A.
"""
"""
We get the maximum likelihood estimate of the probability a[i][j] of a particular transition between states i and j by 
counting the number of times the transition was taken, lets say C(i → j), and then normalizing by the total count of all
times we took any transition from state i:
    a[i][j]=C(i → j)/(Sum(C(i → q))forAll q∈Q)
"""

'\nLet us begin by considering the much simpler case of training a Markov chain rather than a hidden Markov model. Since \nthe states in a Markov chain are observed, we can run the model on the observation sequence and directly see which path \nwe took through the model and which state generated each observation symbol. A Markov chain of course has no emission \nprobabilities B (alternatively, we could view a Markov chain as a degenerate hidden Markov model where all the b proba-\nbilities are 1.0 for the observed symbol and 0 for all other symbols). Thus, the only probabilities we need to train are \nthe transition probability matrix A.\n'

In [3]:
"""
For an HMM, we cannot compute these counts directly from an observation sequence since we don’t know which path of states 
was taken through the machine for a given input. The Baum-Welch algorithm uses two neat intuitions to solve this problem.
The first idea is to iteratively estimate the counts. We will start with an estimate for the transition and observation 
probabilities and then use these estimated probabilities to derive better and better probabilities. The second idea is 
that we get our estimated probabilities by computing the forward probability for an observation and then dividing that 
probability mass among all the different paths that contributed to this forward probability.
"""

'\nFor an HMM, we cannot compute these counts directly from an observation sequence since we don’t know which path of states \nwas taken through the machine for a given input. The Baum-Welch algorithm uses two neat intuitions to solve this problem.\nThe first idea is to iteratively estimate the counts. We will start with an estimate for the transition and observation \nprobabilities and then use these estimated probabilities to derive better and better probabilities. The second idea is \nthat we get our estimated probabilities by computing the forward probability for an observation and then dividing that \nprobability mass among all the different paths that contributed to this forward probability.\n'